# PHYS 210 Mini-Project 03
### Ferromagneticsm
Due Wed, Nov 23, 2022 - 9am

#### Project objective

Explore the Curie temperature and the Ising model of Ferromagnetism

#### Before getting started

* Read the Mini-Project 03 background handout in the same folder as this notebook. It provides all the details.

In [3]:
"""The code in this project will simulate the evolution of
a ferromagnetic system over time using th Ising model.
This simulation will be modelled in 2-dimensional
animations and the Curie temperature will be estimated."""

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
import copy


def interaction(i, j, grid):
    """This function calculates the spin-spin interactions of one electron
    and its non-diagonal neighbours.

    i: x position of electron in grid.
    j: y position of electron in grid.
    grid: the grid model of electrons.

    returns the value of the spin interactions."""

    # calculates the value of these interactions based on the provided formula
    interacval = grid[j, i] * (grid[j, i - 1] + grid[j, (i + 1) % x] +
                               grid[j - 1, i] + grid[(j + 1) % y, i])
    return interacval  # returns the value of spin interactions


def initial_H(grid):
    """This function calculates the initial energy of
    the electron grid model.

    grid: the grid model of electrons.

    returns the energy of the grid of electrons."""

    J = 1  # magnitude of spin-spin interaction
    # defining total energy variable out of loop
    total = 0
    for j in range(y):
        for i in range(x):
            # adds the interactions of electrons in grid
            total += interaction(i, j, grid)
    return -J / 2 * total  # returns the energy of the grid


def Hprime(iflip, jflip, old_energy, grid):
    """This function calculates the energy of a new
    configuration resulting in the flipping of an
    electron's spin at a specified position in the 50x50
    grid. This calculation is based on the previous
    configurations energy and the properties of the
    summation formula for calculating the energy of
    the overall system.

    iflip: the x position of electron to be flipped.
    jflip: the y position of electron to be flipped.
    old_energy: the energy of the previous configuration.
    grid: the grid model of electrons.

    returns the energy of the new (tentative) configuration."""

    J = 1  # magnitude of spin-spin interaction
    # calculates the energy of new configuration
    # this calculation has been simplified algebraically
    newE = old_energy + 2 * J * interaction(iflip, jflip, grid)
    return newE  # returns energy of new configuration


def Pkeep(new_energy, old_energy):
    """This function calculates the probability that a new
    configuration is kept in the case this new configuration
    has higher energy than the origional.

    new_energy: the energy of the new configuration.
    old_energy: the energy of the previous configuration.

    returns the probability of keeping the new configuratrion."""

    # calculates and returns the probability of keeping new config
    return np.e**(-(new_energy - old_energy) / T)


def UpdateState(old_energy, grid):
    """This function randomly chooses an electron in the
    50x50 grid to be flipped. It then calculates if this
    energy if lower or higher, and if it is lower this
    new energy is stored into the global variable
    accepted_energy. If the new energy is higher, the new
    configuration will be kept or discarded based on a
    probablity dependent on the temperature of the system
    and the difference in energy between the 2 configurations.

    old_energy: the energy of the previous configuration.
    grid: the grid model of electrons.

    returns nothing."""

    # referances the global variable accepted_energy
    global accepted_energy

    # position of random electron flipped
    randx = np.random.randint(low=0, high=x)
    randy = np.random.randint(low=0, high=y)

    # calculates the energy of the new configuration
    newE = Hprime(randx, randy, old_energy, grid)

    # if the new configuration has lower energy
    if newE <= old_energy:
        # modify grid in place to accept change
        grid[randy, randx] *= -1
        # accepted energy becomes a copy of newE
        accepted_energy = copy.deepcopy(newE)
    else:  # if the new configuration has higher energy
        # if by chance the configuration is accpeted
        if np.random.random() <= Pkeep(newE, old_energy):
            # modify grid in place to accept change
            grid[randy, randx] *= -1
            # accepted energy becomes a copy of newE
            accepted_energy = copy.deepcopy(newE)
        # if by chance the configuration is rejected
        else:
            # accepted energy is the energy of old config
            accepted_energy = copy.deepcopy(old_energy)


def GetFrames(temperature, stepcount, framecount):
    """This function simulates the ferromagnetic system
    over a specified number of steps at a given temperature
    while storing frames for later animation.

    temperature: the temperature to simulate the syystem at.
    stepcount: the number of iterations in the simulation.
    framecount: the number of frames in the animation

    returns the list of images to be animated."""

    # makes a 50x50 grid of electrons with random state -1 or 1
    # defines x and y as global variables as they will be used
    # in other functions
    global x
    global y
    x = 50  # width of electron grid
    y = 50  # height of electron grid
    # creates a grid of random 0s and 1s with dimensions x by y
    grid = np.random.randint(low=0, high=2, size=(x, y))
    # converts all 0s to -1s
    grid[grid < 1] = -1
    # references global variable T as it is used in other functions
    global T
    T = temperature  # temperature
    steps = stepcount  # number of iterations
    ims = []  # List to store grids of electron states
    frames = framecount  # number of frames for the animation
    finterval = int(steps / frames)  # number of steps between rendered frames

    startE = initial_H(grid)

    for n in range(steps):
        if n == 0:  # checks if it is loops first run
            # uses starting energy as previous energy for state update
            UpdateState(startE, grid)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)
        else:
            # uses the energy of the previous iteration oldE
            # for the state update
            UpdateState(oldE, grid)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)

        # checks if this interval should be a frame in the animation
        if n % finterval == 0:
            # each image added as a tuple to ims
            ims.append((plt.pcolormesh(grid, cmap='jet'), ))
    return ims  # returns the list of images to be animated


def GenEndState(temperature, stepcount):
    """This function generates the ""end state"" for the
    ferromagnetic system after a specified number of steps
    at a given temperature.

    temperature: the temperature to simulate the syystem at.
    stepcount: the number of iterations in the simulation.

    returns the final 50x50 grid of electons after specified
    number of iterations."""

    # makes a 50x50 grid of electrons with random state -1 or 1
    # defines x and y as global variables as they will be used
    # in other functions
    global x
    global y
    x = 50  # width of electron grid
    y = 50  # height of electron grid
    # creates a grid of random 0s and 1s with dimensions x by y
    grid = np.random.randint(low=0, high=2, size=(x, y))
    # converts all 0s to -1s
    grid[grid < 1] = -1
    # references global variable T as it is used in other functions
    global T
    T = temperature  # temperature
    steps = stepcount  # number of iterations

    # calculates starting energy of system
    startE = initial_H(grid)

    for n in range(steps):  # runs for number of desired steps
        if n == 0:  # checks if it is loops first run
            # uses starting energy as previous energy for state update
            UpdateState(startE, grid)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)
        elif n == steps - 1:  # checks if this is last iteration of loop
            endstate = np.copy(grid)  # stores copy of current system
            return endstate  # returns the final state of system
        else:
            # uses the energy of the previous iteration oldE
            # for the state update
            UpdateState(oldE, grid)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)


def Animation(temperature, name, framecount, stepcount):
    """This function saves an animation with a specified name
    and number of frames of the simulated ferromagnetic system
    at a given temperature. It will run a specified number of
    iterations for that system.

    temperature: the temperature to simulate the syystem at.
    name: the name of the file to be saved to the computer.
    framecount: the number of frames in the animation.
    stepcount: the number of iterations in the simulation.

    returns nothing"""

    fig = plt.figure()  # Create a new figure

    # runs the GetFrames() function at the specified temperature, stepcount,
    # and framecount to obtain the frames to be stored in ims for animation
    ims = GetFrames(temperature=temperature,
                    stepcount=stepcount,
                    framecount=framecount)

    # Animate the frames stored in ims
    imani = animation.ArtistAnimation(
        fig,  # Not used here, but would be needed for resizing, etc
        ims,  # The list to animate
        interval=50,  # Time between frames in ms
        repeat=False  # Repeat not used here. Useful in later projects
    )

    # Save to a file: which can be viewed from outside or within the notebook
    imani.save("{}.webm".format(name), extra_args=['-vcodec', 'libvpx'])
    plt.close()  # Prevents a stray plot from appearing
    # Release crucial memory related to these objects
    del imani
    del ims


def getM(temp):
    """This function calculates the average magnetic moment M for
    the end state of the modelled system (after 600,000 iterations)
    at some given temperature.

    temp: the temperature at which the system is modelled at.

    returns the average magnetic moment M for the end state of
    the system."""

    # references global x and y variables (the size of the electron grid)
    global x
    global y
    mewb = 1  # the magnetic moment of an electron used for this project
    # calculates the final grid of electrons after 600,000 iterations
    # this is called the "end state" throuhout this project
    endstate = GenEndState(temp, 600000)
    endsum = np.sum(endstate)  # sums the spins of electrons in grid
    M = endsum / (x * y) * mewb  # calculates average magnetic moment
    return M  # returns average magnetic moment for system


def maxM(n, temp):
    """This function independently calculates n values of the
    average magnetic moment M for the end state of the modelled
    system at a specific temperature and returns the maximum of
    these values (absolute values).

    n: number of times we want to independently calculate M.

    returns the maximum value of M obtained from the n independent
    values."""

    arr = np.array([])  # defines an empty array outside of for loop
    for i in range(n):  # loops n times
        # appends calculated M value to array arr
        arr = np.append(arr, getM(temp))
    arr = np.abs(arr)  # converts the array arr to absolute value
    return np.max(arr)  # returns the maximim value of M in arr

In [6]:
# Generate Deliverable 1 here: Plot of M vs T
# the list of temperatures to be tested
T_arr = [0.01, 0.1, 1, 2, 3, 4, 5, 10, 100]

# uses map() function to map T_arr and number of iterations to mxM()
M_arr = list(map(maxM, [5] * len(T_arr), T_arr))

# plots M vs T for T_arr
plt.figure(1)
plt.plot(T_arr, M_arr)
# makes the x axis log scale
plt.xscale('log')
# labels the x and y axis appropriately
plt.xlabel('Temperature (K)')
plt.ylabel('Average Magnetic Moment (J/T)')

# calculates and prints the estimated Curie temperature
Tc = 1.8
print("The estimated Curie temperature for this ferromagnetic system is: {} K".
      format(Tc))

KeyboardInterrupt: 

In [5]:
# Generate Deliverable 2.1 here: Animation for T = 0.1
# runs the predefined animation function to save an animated video
Animation(0.1, 'ferro0.1', 200, 600000)
# Link the video into the notebook
HTML(
    '<video controls> <source src="ferro0.1.webm" type="video/webm"> </video>')

In [6]:
# Generate Deliverable 2.2 here: Animation for T = 2.5
# runs the predefined animation function to save an animated video
Animation(2.5, 'ferro2.5', 200, 600000)
# Link the video into the notebook
HTML(
    '<video controls> <source src="ferro2.5.webm" type="video/webm"> </video>')

In [7]:
# Generate Deliverable 2.2 here: Animation for T = 2.5
# runs the predefined animation function to save an animated video
Animation(100, 'ferro100', 200, 600000)
# Link the video into the notebook
HTML(
    '<video controls> <source src="ferro100.webm" type="video/webm"> </video>')

# Acknowledgements

In the cell below, please describe the role of **anyone other than yourself** who contributed to the work shown in this notebook.

Its ok to get help from us and classmates! Please get in the habit of acknowledging such contributions.

If you want to refer to a classmate, please use only their cocalc email-id and not their name - or you could just say something like: "a classmate gave me the idea to use xxx feature to solve yyy problem."


_Acknowledgements here:_ Matplotlib documentation was heavily utilized in the creation of the 3d animation for the extension.


# Extension Code and Description
All solution code for the main project question should appear in the cell "cell-project1-main" above. Project extensions go in the cell "cell-extension" immediately below and the descriptions of your extension go in the cell below that.

In [3]:
# OPTIONAL project extension here
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import copy


def interaction3d(i, j, k, voxel):
    """This function calculates the spin-spin interactions of one electron
    and its non-diagonal neighbours.

    i: x position of electron in voxel.
    j: y position of electron in voxel.
    k: z position of electron in voxel.
    voxel: the 3D model of electrons.

    returns the value of the spin interactions."""

    # calculates the value of these interactions based on the provided formula
    interacval = voxel[k, j, i] * (
        voxel[k, j, i - 1] + voxel[k, j, (i + 1) % x] + voxel[k, j - 1, i] +
        voxel[k,
              (j + 1) % y, i] + voxel[k - 1, j, i] + voxel[(k + 1) % z, j, i])
    return interacval  # returns the value of spin interactions


def initial_H3d(voxel):
    """This function calculates the initial energy of
    the electron voxel model.

    voxel: the 3D model of electrons.

    returns the energy of the voxel of electrons."""

    J = 1  # magnitude of spin-spin interaction
    # defining total energy variable out of loop
    total = 0
    for k in range(z):
        for j in range(y):
            for i in range(x):
                # adds the interactions of electrons in voxel
                total += interaction3d(i, j, k, voxel)
    return -J / 2 * total  # returns the energy of the voxel


def Hprime3d(iflip, jflip, kflip, old_energy, voxel):
    """This function calculates the energy of a new
    configuration resulting in the flipping of an
    electron's spin at a specified position in the
    10x10x10 voxel. This calculation is based on the
    previous configurations energy and the properties
    of the summation formula for calculating the energy
    of the overall system.

    iflip: the x position of electron to be flipped.
    jflip: the y position of electron to be flipped.
    kflip: the z position of electron to be flipped.
    old_energy: the energy of the previous configuration.
    voxel: the voxel model of electrons.

    returns the energy of the new (tentative) configuration."""

    J = 1  # magnitude of spin-spin interaction
    # calculates the energy of new configuration
    # this calculation has been simplified by algebra
    newE = old_energy + 2 * J * interaction3d(iflip, jflip, kflip, voxel)
    return newE  # returns energy of new configuration


def Pkeep(new_energy, old_energy):
    """This function calculates the probability that a new
    configuration is kept in the case this new configuration
    has higher energy than the origional.

    new_energy: the energy of the new configuration.
    old_energy: the energy of the previous configuration.

    returns the probability of keeping the new configuratrion."""

    # calculates and returns the probability of keeping new config
    return np.e**(-(new_energy - old_energy) / T)


def UpdateState3d(old_energy, voxel):
    """This function randomly chooses an electron in the
    10x10x10 voxel to be flipped. It then calculates if this
    energy if lower or higher, and if it is lower this
    new energy is stored into the global variable
    accepted_energy. If the new energy is higher, the new
    configuration will be kept or discarded based on a
    probablity dependent on the temperature of the system
    and the difference in energy between the 2 configurations.

    old_energy: the energy of the previous configuration.
    voxel: the voxel model of electrons.

    returns nothing."""

    # referances the global variable accepted_energy
    global accepted_energy

    # position of random electron flipped
    randx = np.random.randint(low=0, high=x)
    randy = np.random.randint(low=0, high=y)
    randz = np.random.randint(low=0, high=z)

    # calculates the energy of the new configuration
    newE = Hprime3d(randx, randy, randz, old_energy, voxel)

    # if the new configuration has lower energy
    if newE <= old_energy:
        # modify voxel in place to accept change
        voxel[randz, randy, randx] *= -1
        # accepted energy becomes a copy of newE
        accepted_energy = copy.deepcopy(newE)
    else:  # if the new configuration has higher energy
        # if by chance the configuration is accpeted
        if np.random.random() <= Pkeep(newE, old_energy):
            # modify voxel in place to accept change
            voxel[randz, randy, randx] *= -1
            # accepted energy becomes a copy of newE
            accepted_energy = copy.deepcopy(newE)
        # if by chance the configuration is rejected
        else:
            # accepted energy is the energy of old config
            accepted_energy = copy.deepcopy(old_energy)


def GetFrames3d(temperature, stepcount, framecount):
    """This function simulates the ferromagnetic system
    over a specified number of steps at a given temperature
    while storing frames for later animation.

    temperature: the temperature to simulate the syystem at.
    stepcount: the number of iterations in the simulation.
    framecount: the number of frames in the animation.

    returns the list of images to be animated."""

    # makes a 10x10x10 voxel of electrons with random state -1 or 1
    # defines x, y and z as global variables as they will be used
    # in other functions
    global x
    global y
    global z
    x = 13  # width of voxel
    y = 13  # height of voxel
    z = 13  # depth of voxel

    # creates a grid of random 0s and 1s with dimensions x by y by z
    voxel = np.random.randint(low=0, high=2, size=(x, y, z))
    # converts all 0s to -1s
    voxel[voxel < 1] = -1
    # references global variable T as it is used is other functions
    global T
    T = temperature  # temperature
    steps = stepcount  # number of iterations
    frames = framecount  # number of frames for the animation
    finterval = int(steps / frames)  # number of steps between rendered frames
    ims = []

    startE = initial_H3d(voxel)

    for n in range(steps):
        if n == 0:  # checks if it is loops first run
            # uses starting energy as previous energy for state update
            UpdateState3d(startE, voxel)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)
        else:
            # uses the energy of the previous iteration oldE
            # for the state update
            UpdateState3d(oldE, voxel)
            # assigns a copy of the accepted energy to oldE for the
            # next iteration of the for loop where it will be used
            # as the previous energy
            oldE = copy.deepcopy(accepted_energy)

        # checks if this interval should be a frame in the animation
        if n % finterval == 0:
            # each electron state voxel added to ims
            statearr = voxel
            voxelarr = (statearr == 1)
            ims.append(voxelarr)
    return ims  # returns the list of electron states to be animated


def Animation3d(name, temperature, stepcount, framecount):
    """This function creates a 3 dimensional animation simulating
    a ferromagnetic system. Electrons with spin=1 are displayed in
    coloured cubes while electrons with spin=-1 are transparent.

    name: the name of the file to be saved to the drive.
    temperature: the temperature to simulate the syystem at.
    stepcount: the number of iterations in the simulation.
    framecount: the number of frames in the animation.

    """

    # runs the GetFrames3d() function at the specified temperature, stepcount,
    # and framecount to obtain the frames to be stored in ims for animation
    ims = GetFrames3d(temperature=temperature,
                      stepcount=stepcount,
                      framecount=framecount)

    # creates a new figure
    fig = plt.figure()
    # projects figure into 3D
    ax = plt.axes(projection='3d')

    # code for colouring voxels
    r, g, b = np.indices((x + 1, y + 1, z + 1)) / (x - 1)

    # code for colouring voxels
    def midpoints(x):

        sl = ()
        for i in range(x.ndim):
            x = (x[sl + np.index_exp[:-1]] + x[sl + np.index_exp[1:]]) / 2
            sl += np.index_exp[:]
        return x

    # code for colouring voxels
    rc = midpoints(r)
    gc = midpoints(g)
    bc = midpoints(b)

    # code for colouring voxels
    colors = np.zeros(ims[0].shape + (3, ))
    colors[..., 0] = rc / 1.5 + 0.15
    colors[..., 1] = gc / 4 + 0.1
    colors[..., 2] = bc / 1.5 + 0.15

    def frame(i):
        """Function to be used in FuncAnimation

        returns the figure for the corresponding electron
        state stored in the list ims"""

        ax.clear()

        voxelarr = ims[i]

        ax.voxels(voxelarr, facecolors=colors, edgecolor='k', linewidth=0.1)
        return fig

    # Animate the frames stored in ims
    ani3d = animation.FuncAnimation(fig,
                                    frame,
                                    frames=framecount,
                                    blit=False,
                                    interval=90)
    # Save to a file: which can be viewed from outside or within the notebook
    ani3d.save("{}.webm".format(name), extra_args=['-vcodec', 'libvpx'])
    plt.close()

In [4]:
temperature = 0.1
stepcount = 75000
framecount = 60

Animation3d('ferro0.1 3d', temperature, stepcount, framecount)

HTML(
    '<video controls> <source src="ferro0.1 3d.webm" type="video/webm"> </video>')

_In this cell, please describe any new language features or project extension you have implemented:_ My extension is essentially the expansion of the 2 dimensional system from the main project into 3 dimensions. I implemented the use of a 3 dimentional plotting function within matplotlib called voxels which I used to model a 3 dimensional electron lattice. I also utilized FuncAnimation which allowed me to model the evolution of this system with time assuming it followed similar laws as the main project's 2D system. In my main project I also utilized a map function to map the temperature values into my GenEndState function.



# Grading cells
The cells below marked as "grade use only" are created as placeholders so that we can provide a manual grade and comments for each category. 

Exceptions are the "2. Style" test, which has an associated autograder test that you can run to check style and the timing cell "cell-optimization0", which you can use to test your code execution time.

In [10]:
# 1. Code execution (grader use only)

In [11]:
# 2. Style: pep8 (see note below regarding use of the Format button to fix many errors)
#
# Tests for pep8 returns warnings or errors. You may need to hit 'Save' after making changes for them to take effect.
nb_name = "project03.ipynb"
cells_to_check = []
stop_at = ['cell-extension']
# check_style2.py uses cells_to_check and nb_name
%run -i check_style2.py

checking cell: cell-project3-1
checking cell: cell-project3-2
checking cell: cell-project3-3
checking cell: cell-project3-4
checking cell: cell-project3-5


Also note that you can use the Format button while in a code cell to automagically fix most pep8 errors (other than way too long print statements)

![](project02-format.png)

In [12]:
# 3. Results (grader use only)

In [13]:
# 4. Readability (grader use only)


In [14]:
# 5. Plot (grader use only)

In [15]:
# Check execution time
nb_name = "project03.ipynb"
cells_to_time = []
stop_at = ['cell-extension']
%run -i time_cells2.py

Time for cell: cell-project3-1 time: 0.02


KeyboardInterrupt: 

Time for cell: cell-project3-2 time: 54.64


Time for cell: cell-project3-3 time: 10.70


Time for cell: cell-project3-4 time: 10.66


Time for cell: cell-project3-5 time: 10.55
Total time: 86.56


In [ ]:
# 5. Code optimization/timing (grader use only)


In [ ]:
# B2. New Functionality/Language features (grader use only)